In [ ]:
!curl -LsSf https://astral.sh/uv/install.sh | sh

In [ ]:
!mkdir hands_on_genai
!cd hands_on_genai && uv init && uv add transformers && uv add sentence_transformers && uv add gensim

In [ ]:
from transformers import AutoModel, AutoTokenizer

# 加载分词器
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-base")

# 加载语言模型
model = AutoModel.from_pretrained("microsoft/deberta-v3-xsmall")

#对句子进行分词
tokens = tokenizer("Hello World", return_tensors='pt')

# 处理词元
output = model(**tokens)[0]

print(output.shape)
for token in tokens['input_ids'][0]:
    print(tokenizer.decode(token))


In [ ]:
from sentence_transformers import SentenceTransformer

# 加载模型
model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

# 将文本转换为文本嵌入
vector = model.encode("Best movie ever!")

print(vector.shape)

In [ ]:
!pip install gensim
!pip install pandas

In [ ]:
import gensim.downloader as api

# glove, 训练数据来自维基百科
# 其它选项包括 word2vec-google-news-300
model = api.load("glove-wiki-gigaword-50")
model.most_similar([model['king']], topn=11)

In [ ]:
# 歌曲推荐 Embedding
import pandas as pd
from urllib import request

# 获取播放列表数据集文件
data = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/train.txt')

# 解析播放列表数据集文件。跳过前两行，因为它们只包含元数据
lines = data.read().decode('utf-8').split('\n')[2:]
playlists = [s.rstrip().split() for s in lines if len(s.split()) > 1]

# 加载歌曲元数据
songs_file = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/song_hash.txt')
songs_file = songs_file.read().decode('utf-8').split('\n')
songs = [s.rstrip().split('\t') for s in songs_file]
songs_df = pd.DataFrame(data=songs, columns=['id', 'title', 'artist'])
songs_df = songs_df.set_index('id')

print('Playlist #1:\n ', playlists[0], '\n')
print('Playlist #2:\n ', playlists[1])

In [ ]:
from gensim.models import Word2Vec

# 训练我们的 word2vec 模型
model = Word2Vec(
    playlists,
    vector_size=32,
    window=20,
    negative=50,
    min_count=1,
    workers=4
)

In [ ]:
song_id = 2172
print(songs_df.iloc[2172])
 # 让模型找出与歌曲 2172 相似的歌曲
model.wv.most_similar(positive=str(song_id))

In [ ]:
import numpy as np

def print_recommendations(song_id):
  similar_songs = np.array(
      model.wv.most_similar(positive=str(song_id), topn=5)
  )[:, 0]

  return songs_df.iloc[similar_songs]

print_recommendations(2172)